In [1]:
# Read data from pdf

from pypdf import PdfReader

def extract_text(path):
  reader = PdfReader(path)
  text = ''

  for page in reader.pages:
    text += page.extract_text() + '\n'
  return text

In [3]:
text = extract_text('The Finance Bill 2025.pdf')
text

'   \nSPECIAL ISSUE \nNATION I. \'01.INCI I.FOR \nk P OR TI NC \nLIPRAPY  \n \n   \nKenya Gazette Supplement No. 63 (National Assembly Bills No. 19) \nREPUBLIC OF KENYA \nKENYA GAZETTE SUPPLEMENT \nNATIONAL ASSEMBLY BILLS, 2025 \nNAIROBI, 6th May, 2025 \nCONTENT \nBill for Introduction into the National Assembly— \nPAGE \nThe Finance Bill, 2025  \t 335 \nNATIONAL COUNCIL FOR \nLAW REPORTING \n0 9 MAY 2025 \nLIBRARY ARCHIVE \nPRINTED AND PUBLISHED BY THE GOVERNMENT PRINTER, NAIROBI \n335 \nTHE FINANCE BILL, 2025 \nA Bill for \nAN ACT of Parliament to amend the laws relating to \nvarious taxes and duties; and for matters incidental \nthereto \nENACTED by the Parliament of Kenya, as follows — \nPART I--PRELIMINARY \n1. This Act may be cited as the Finance Act, 2025 \nand shall come into operation as follows — \n(a) sections 12 and 56, on the 1st of January, 2026; \nand \n(b) all other sections, on 1st July, 2025. \nPART II—INCOME TAX \n2. Section 2 of the Income Tax Act is amended—\n(a) i

In [4]:
def chunk_text(text, size=800):
  words = text.split()
  chunks=[]
  for i in range(0, len(words), size):
    chunks.append(" ".join(words[i:i+size]))
  return chunks

chunks = chunk_text(text)

In [5]:
chunks

['SPECIAL ISSUE NATION I. \'01.INCI I.FOR k P OR TI NC LIPRAPY Kenya Gazette Supplement No. 63 (National Assembly Bills No. 19) REPUBLIC OF KENYA KENYA GAZETTE SUPPLEMENT NATIONAL ASSEMBLY BILLS, 2025 NAIROBI, 6th May, 2025 CONTENT Bill for Introduction into the National Assembly— PAGE The Finance Bill, 2025 335 NATIONAL COUNCIL FOR LAW REPORTING 0 9 MAY 2025 LIBRARY ARCHIVE PRINTED AND PUBLISHED BY THE GOVERNMENT PRINTER, NAIROBI 335 THE FINANCE BILL, 2025 A Bill for AN ACT of Parliament to amend the laws relating to various taxes and duties; and for matters incidental thereto ENACTED by the Parliament of Kenya, as follows — PART I--PRELIMINARY 1. This Act may be cited as the Finance Act, 2025 and shall come into operation as follows — (a) sections 12 and 56, on the 1st of January, 2026; and (b) all other sections, on 1st July, 2025. PART II—INCOME TAX 2. Section 2 of the Income Tax Act is amended— (a) in subsection (1)— (i) in the definition of "debenture", by deleting the expression

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('BAAI/bge-small-en-v1.5')

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [7]:
finance_embeddings = model.encode(chunks).tolist()

finance_embeddings

[[-0.09196730703115463,
  -0.016723837703466415,
  0.06988348811864853,
  -0.005943676456809044,
  0.04793059825897217,
  -0.004258892498910427,
  0.03248222917318344,
  -0.01627930998802185,
  -0.02153024822473526,
  0.024296047165989876,
  0.05288318917155266,
  0.04042564705014229,
  0.017774881795048714,
  -0.002317194128409028,
  0.0005365957622416317,
  -0.07159734517335892,
  -0.032707493752241135,
  -0.10074988007545471,
  -0.04474719241261482,
  0.1026776134967804,
  0.1051274985074997,
  -0.015476631000638008,
  0.036192093044519424,
  0.03144374117255211,
  0.06897581368684769,
  0.05275843292474747,
  -0.00747136352583766,
  -0.05458301305770874,
  0.025479203090071678,
  -0.1927396059036255,
  0.0064645432867109776,
  -0.02287677861750126,
  0.010967377573251724,
  0.002314050914719701,
  0.0010949914576485753,
  -0.01732758618891239,
  -0.03197813034057617,
  0.015344031155109406,
  0.011496610008180141,
  0.014794327318668365,
  -0.008045506663620472,
  -0.00781341362744

In [8]:
len(finance_embeddings)

54

In [9]:
import chromadb

client = chromadb.PersistentClient(path='./chromadb')

In [10]:
collection = client.create_collection(
    name='kenyan_finance_bills',
    metadata={'description':'Kenyan Finance Bills for the "Wantam" era'}
)

print('The collection has been created:', collection.name)

The collection has been created: kenyan_finance_bills


In [11]:
ids = [f"doc_{i}" for i in range(len(chunks))]

ids

['doc_0',
 'doc_1',
 'doc_2',
 'doc_3',
 'doc_4',
 'doc_5',
 'doc_6',
 'doc_7',
 'doc_8',
 'doc_9',
 'doc_10',
 'doc_11',
 'doc_12',
 'doc_13',
 'doc_14',
 'doc_15',
 'doc_16',
 'doc_17',
 'doc_18',
 'doc_19',
 'doc_20',
 'doc_21',
 'doc_22',
 'doc_23',
 'doc_24',
 'doc_25',
 'doc_26',
 'doc_27',
 'doc_28',
 'doc_29',
 'doc_30',
 'doc_31',
 'doc_32',
 'doc_33',
 'doc_34',
 'doc_35',
 'doc_36',
 'doc_37',
 'doc_38',
 'doc_39',
 'doc_40',
 'doc_41',
 'doc_42',
 'doc_43',
 'doc_44',
 'doc_45',
 'doc_46',
 'doc_47',
 'doc_48',
 'doc_49',
 'doc_50',
 'doc_51',
 'doc_52',
 'doc_53']

In [12]:
len(chunks) == len(ids)

True

In [13]:
collection.add(
    documents=chunks,
    embeddings=finance_embeddings,
    ids=ids
)

In [14]:
collection.count()

54

In [15]:
query = 'What does the Finance bill 2025 say about taxes'

In [16]:
query_embedding = model.encode(query).tolist()
query_embedding

[-0.08010197430849075,
 -0.05470728874206543,
 0.023225873708724976,
 -0.0005235627759248018,
 0.04822197183966637,
 -0.015666309744119644,
 0.014325914904475212,
 0.009607546031475067,
 0.035056039690971375,
 -0.011964265257120132,
 0.025990663096308708,
 0.046274762600660324,
 -0.025720596313476562,
 0.043116308748722076,
 -0.05770903080701828,
 -0.014119315892457962,
 0.01722518354654312,
 -0.06831812858581543,
 -0.09750066697597504,
 0.05060607194900513,
 0.11671382933855057,
 0.0106645617634058,
 0.021610666066408157,
 -0.022749722003936768,
 0.02034614607691765,
 -0.013548986986279488,
 -0.012385009787976742,
 -0.016918931156396866,
 -0.05719207599759102,
 -0.1295880824327469,
 0.03021944873034954,
 -0.08324073255062103,
 -0.054742928594350815,
 -0.030500849708914757,
 0.07293257117271423,
 -0.040564749389886856,
 -0.014917395077645779,
 0.02329017035663128,
 0.021627936512231827,
 0.05669010430574417,
 0.004145924001932144,
 0.05283868685364723,
 -0.04686415567994118,
 0.0284052

In [17]:
results = collection.query(
    query_embeddings=query_embedding,
    n_results=3
)

results

{'ids': [['doc_51', 'doc_0', 'doc_45']],
 'embeddings': None,
 'documents': [['460 The Finance Bill, 2025 (i) the names and addresses of each person to whom a payment was made; (ii) where the payment is for services, the amount of the payment specifying whether the payment is a commission of any kind or is for expenses incurred in connection with rendering the services; (iii) where the payment is in any form of valuable consideration other than money, the particulars of the consideration; and (iv) such other particulars as the Commissioner may specify; (b) references to payments for services include references to payments in the nature of commission of any kind and references to payments in respect of expenses incurred in connection with the rendering of services; and (c) references to the making of payments include references to the giving of any form of valuable consideration, and the requirement imposed by paragraph (a)(iii) to state the amount of a payment shall, in relation to any